In [30]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9eac131ca2e04a739138b3d5f5701a44 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='fz-k2ak1WrGTL_1xdnqxbxWG5uNlO-WfQvDcfWS7l3m8',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_9eac131ca2e04a739138b3d5f5701a44.get_object(Bucket='module8-donotdelete-pr-zozbishnggzy75',Key='coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

neighborhoods = pd.read_csv(body)
neighborhoods.head(10)


,Borough,Neighbourhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790
5,Central Toronto,Davisville North,43.712751,-79.390197
6,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,Central Toronto,Davisville,43.704324,-79.388790
8,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values





In [3]:
!pip install folium

     |████████████████████████████████| 94 kB 5.7 MB/s  eta 0:00:01


In [4]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [47]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


This is only Borough which contains Toronto

In [48]:
westtoronto_data = neighborhoods[neighborhoods['Borough'] == 'West Toronto'].reset_index(drop=True)
westtoronto_data.head(10)

,Borough,Neighbourhood,Latitude,Longitude
0,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
1,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
3,West Toronto,"High Park, The Junction South",43.661608,-79.464763
4,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
5,West Toronto,"Runnymede, Swansea",43.651571,-79.484450


Creating a map of West Toronto using Latitude and Logitude values

In [39]:

import folium 
#latitude = '43.6606'
#longitude = '79.4692'

map_westtoronto = folium.Map(location=[latitude, longitude], zoom_start=3)

# add markers to map
for lat, lng, label in zip(westtoronto_data['Latitude'], westtoronto_data['Longitude'], westtoronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_westtoronto)  
    
map_westtoronto

In [49]:
CLIENT_ID = '3SHIHVNX2RDNHOZAE0IYPHBGIGGUNMYJSXCM20VTPY4OWIZ2' # your Foursquare ID
CLIENT_SECRET = 'FZG13JSC2IN5XHQROCJQTMBSPH3PIKNGC5JCZP3QAFGRAXUI' # your Foursquare Secret
ACCESS_TOKEN = 'NVAO2NQCRYPOJXSOHEHVLFIGMRHHJ4RETIKQLBNGZ40FVGC3' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value



Exploring the first Neighbourhood name

In [50]:
westtoronto_data.loc[0, 'Neighbourhood']

'Dufferin, Dovercourt Village'

Get the neighborhood's latitude and longitude values.

In [51]:
neighborhood_latitude = westtoronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = westtoronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = westtoronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Dufferin, Dovercourt Village are 43.66900510000001, -79.4422593.


Now, let's get the top 100 venues that are in Duffering, Dovercourt within a radius of 500 meters.

In [52]:
LIMIT = 10 # limit of number of venues returned by Foursquare API


radius = 1000 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=3SHIHVNX2RDNHOZAE0IYPHBGIGGUNMYJSXCM20VTPY4OWIZ2&client_secret=FZG13JSC2IN5XHQROCJQTMBSPH3PIKNGC5JCZP3QAFGRAXUI&v=20180605&ll=43.66900510000001,-79.4422593&radius=1000&limit=10'

Send the GET request and examine the resutls

In [53]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600b40c6095fbc19cb884629'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Davenport',
  'headerFullLocation': 'Davenport, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 69,
  'suggestedBounds': {'ne': {'lat': 43.67800510900002,
    'lng': -79.42984025053214},
   'sw': {'lat': 43.660005090999995, 'lng': -79.45467834946787}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5753753b498eeb535c53aed5',
       'name': 'The Greater Good Bar',
       'location': {'address': '229 Geary St',
        'crossStreet': 'at Dufferin St',
        'lat': 43.669409,
        'lng': -79.439267,
        'labeledLatLngs': [{'label': 'disp

In [54]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [55]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,The Greater Good Bar,Bar,43.669409,-79.439267
1,Parallel,Middle Eastern Restaurant,43.669516,-79.438728
2,Blood Brothers Brewing,Brewery,43.669944,-79.436533
3,Balzac’s Coffee,Coffee Shop,43.670042,-79.448342
4,The Sovereign,Café,43.673116,-79.440265


And how many venues were returned by Foursquare?

In [56]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


Explore Neighborhoods in Manhattan and create new venue data frame

In [57]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [58]:
westtoronto_venues = getNearbyVenues(names=westtoronto_data['Neighbourhood'],
                                   latitudes=westtoronto_data['Latitude'],
                                   longitudes=westtoronto_data['Longitude']
                                  )

Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea


Let's check our new dataframe

In [59]:
print(westtoronto_venues.shape)
westtoronto_venues.head()

(60, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dufferin, Dovercourt Village",43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,"Dufferin, Dovercourt Village",43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,"Dufferin, Dovercourt Village",43.669005,-79.442259,Blood Brothers Brewing,43.669944,-79.436533,Brewery
3,"Dufferin, Dovercourt Village",43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery
4,"Dufferin, Dovercourt Village",43.669005,-79.442259,FreshCo,43.667918,-79.440754,Grocery Store


Let's check how many venues were returned for each neighborhood

In [60]:
westtoronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Brockton, Parkdale Village, Exhibition Place",10,10,10,10,10,10
"Dufferin, Dovercourt Village",10,10,10,10,10,10
"High Park, The Junction South",10,10,10,10,10,10
"Little Portugal, Trinity",10,10,10,10,10,10
"Parkdale, Roncesvalles",10,10,10,10,10,10
"Runnymede, Swansea",10,10,10,10,10,10



Let's find out how many unique categories can be curated from all the returned venues

In [61]:
print('There are {} uniques categories.'.format(len(westtoronto_venues['Venue Category'].unique())))

There are 41 uniques categories.


Analyze Each Neighborhood

In [62]:
# one hot encoding
westtor_onehot = pd.get_dummies(westtoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
westtor_onehot['Neighborhood'] = westtoronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [westtor_onehot.columns[-1]] + list(westtor_onehot.columns[:-1])
westtor_onehot = westtor_onehot[fixed_columns]

westtor_onehot.head(10)

,Neighborhood,Antique Shop,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Burrito Place,Café,Coffee Shop,Cuban Restaurant,Dessert Shop,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fish & Chips Shop,Flea Market,Furniture / Home Store,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Ice Cream Shop,Italian Restaurant,Korean Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,New American Restaurant,Park,Pet Store,Pizza Place,Pub,Restaurant,Speakeasy,Supermarket,Sushi Restaurant
0,"Dufferin, Dovercourt Village",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Dufferin, Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,"Dufferin, Dovercourt Village",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Dufferin, Dovercourt Village",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Dufferin, Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"Dufferin, Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,"Dufferin, Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"Dufferin, Dovercourt Village",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"Dufferin, Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,"Dufferin, Dovercourt Village",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [63]:
westtoronto_grouped = westtor_onehot.groupby('Neighborhood').mean().reset_index()
westtoronto_grouped

,Neighborhood,Antique Shop,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Burrito Place,Café,Coffee Shop,Cuban Restaurant,Dessert Shop,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fish & Chips Shop,Flea Market,Furniture / Home Store,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Ice Cream Shop,Italian Restaurant,Korean Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,New American Restaurant,Park,Pet Store,Pizza Place,Pub,Restaurant,Speakeasy,Supermarket,Sushi Restaurant
0,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.1,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
1,"Dufferin, Dovercourt Village",0.0,0.0,0.0,0.2,0.1,0.1,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0
2,"High Park, The Junction South",0.1,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0
3,"Little Portugal, Trinity",0.0,0.0,0.1,0.0,0.0,0.1,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0
4,"Parkdale, Roncesvalles",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
5,"Runnymede, Swansea",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.2


Let's print each neighborhood along with the top 5 most common venues

In [64]:
num_top_venues = 5
for hood in westtoronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = westtoronto_grouped[westtoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop   0.2
1  Furniture / Home Store   0.1
2          Breakfast Spot   0.1
3                     Gym   0.1
4      Italian Restaurant   0.1


----Dufferin, Dovercourt Village----
                       venue  freq
0                     Bakery   0.2
1                       Café   0.1
2                    Brewery   0.1
3  Middle Eastern Restaurant   0.1
4                Music Venue   0.1


----High Park, The Junction South----
                venue  freq
0        Antique Shop   0.1
1                Café   0.1
2           Speakeasy   0.1
3                Park   0.1
4  Italian Restaurant   0.1


----Little Portugal, Trinity----
               venue  freq
0   Asian Restaurant   0.1
1     Ice Cream Shop   0.1
2                Bar   0.1
3         Beer Store   0.1
4  Korean Restaurant   0.1


----Parkdale, Roncesvalles----
                         venue  freq
0                    Gift Shop 

 let's write a function to sort the venues in descending order.

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = westtoronto_grouped['Neighborhood']

for ind in np.arange(westtoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(westtoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(100)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Furniture / Home Store,Italian Restaurant,Bakery,Pet Store,Bar,Breakfast Spot,Gym,Café,Eastern European Restaurant
1,"Dufferin, Dovercourt Village",Bakery,Middle Eastern Restaurant,Café,Supermarket,Bank,Bar,Music Venue,Grocery Store,Brewery,Dog Run
2,"High Park, The Junction South",Furniture / Home Store,Italian Restaurant,Arts & Crafts Store,Bar,Café,Flea Market,Gastropub,Antique Shop,Speakeasy,Park
3,"Little Portugal, Trinity",Asian Restaurant,Pizza Place,Cuban Restaurant,Bar,New American Restaurant,Beer Store,Greek Restaurant,Korean Restaurant,Brewery,Ice Cream Shop
4,"Parkdale, Roncesvalles",Gift Shop,Eastern European Restaurant,Dog Run,Restaurant,Dessert Shop,Cuban Restaurant,Coffee Shop,Movie Theater,Italian Restaurant,Brewery
5,"Runnymede, Swansea",Sushi Restaurant,Bookstore,Coffee Shop,Café,Italian Restaurant,Burrito Place,Fish & Chips Shop,Falafel Restaurant,Pub,Beer Store


Run k-means to cluster the neighborhood into 5 clusters.

In [67]:
kclusters = 5

westtoronto_grouped_clustering = westtoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(westtoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 3, 0, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [68]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

westtoronto_merged = westtoronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
westtoronto_merged = westtoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

westtoronto_merged.head(100) # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Bakery,Middle Eastern Restaurant,Café,Supermarket,Bank,Bar,Music Venue,Grocery Store,Brewery,Dog Run
1,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,3,Asian Restaurant,Pizza Place,Cuban Restaurant,Bar,New American Restaurant,Beer Store,Greek Restaurant,Korean Restaurant,Brewery,Ice Cream Shop
2,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,2,Coffee Shop,Furniture / Home Store,Italian Restaurant,Bakery,Pet Store,Bar,Breakfast Spot,Gym,Café,Eastern European Restaurant
3,West Toronto,"High Park, The Junction South",43.661608,-79.464763,2,Furniture / Home Store,Italian Restaurant,Arts & Crafts Store,Bar,Café,Flea Market,Gastropub,Antique Shop,Speakeasy,Park
4,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Gift Shop,Eastern European Restaurant,Dog Run,Restaurant,Dessert Shop,Cuban Restaurant,Coffee Shop,Movie Theater,Italian Restaurant,Brewery
5,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,4,Sushi Restaurant,Bookstore,Coffee Shop,Café,Italian Restaurant,Burrito Place,Fish & Chips Shop,Falafel Restaurant,Pub,Beer Store


Finally, let's visualize the resulting clusters

In [44]:
import folium
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=3)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]






# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(westtoronto_merged['Latitude'], westtoronto_merged['Longitude'], westtoronto_merged['Neighbourhood'], westtoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters